In [163]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import re
import csv
import time
import random
from datetime import date

def ChromeDriver(url):
    chrome_path= "/usr/local/bin/chromedriver"
    # These are preliminary settings on our chromedriver, we want to make sure it's a maximized browser, disable any
    # infobars and extensions.

    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized");
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")

    # This line of code now actually declares the driver, it has two arguments 'executable_path' where we feed in our
    # directory path for the chromedriver and the 'options' which are the chromedriver settings from above.

    driver = webdriver.Chrome(executable_path=chrome_path, options=options)
    driver.get(url)
    # We declare the url and execute the driver to fetch the URL
    return driver

In [164]:
masterData = pd.read_csv('masterData.csv')
masterData.head()

In [165]:
masterLinks = ['https://github.com/nychealth/coronavirus-data/commits/master/by-race.csv']
def crawler(url):
    # this function will crawl through all of the next pages of the commit history for the by-race csv, and collect the link for each page
    global masterLinks
    html = urlopen(url)
    bs = BeautifulSoup(html, 'html.parser')
    nextPage = bs.find(class_="btn btn-outline BtnGroup-item",text='Older')
    if nextPage.has_attr('href') and nextPage['href'] not in masterLinks:
        link = nextPage['href']
        masterLinks = masterLinks + [link]
        return crawler(link)
    else:
        return

crawler('https://github.com/nychealth/coronavirus-data/commits/master/by-race.csv')

https://github.com/nychealth/coronavirus-data/commits/master?after=b671e0ed09b458f148110ad0d71479cf62580ea4+34&branch=master&path%5B%5D=by-race.csv

https://github.com/nychealth/coronavirus-data/commits/master?after=b671e0ed09b458f148110ad0d71479cf62580ea4+69&branch=master&path%5B%5D=by-race.csv

https://github.com/nychealth/coronavirus-data/commits/master?after=b671e0ed09b458f148110ad0d71479cf62580ea4+104&branch=master&path%5B%5D=by-race.csv

https://github.com/nychealth/coronavirus-data/commits/master?after=b671e0ed09b458f148110ad0d71479cf62580ea4+139&branch=master&path%5B%5D=by-race.csv

success!


In [166]:
days = list()
commits = list()

catch = ''
for masterLink in masterLinks:
    # iterate through all of the links found with the crawler function
    global days
    global commits
    html = urlopen(masterLink)
    bs = BeautifulSoup(html, 'html.parser')

    days = days + bs.find_all(class_="f5 text-normal")
    # collect the commit upload dates in a global list called days

    commits = commits + bs.find_all('clipboard-copy', class_="btn btn-outline BtnGroup-item")
    # collects all of the commit addresses/values into a global list called commits by collecting the 
    # commits off of the specific file page, instead of the master page for the entire NYC dataset, we ensure that each 
    # commit actually updated the data we wanted (i.e. doesn't correspond to a different file or README).
    # Additionally, these commit addresses can be used for both the probable-confirmed file, as well as the master by race      # files.

    if len(days) != len(commits) and catch == '':
        # collects the masterLink on which a specific date had multiple uploads, which lead to unequal lengths of each list
        catch = masterLink

if len(commits) != len(days):
    print('DUPLICATE COMMITS FOUND AT: ' + catch)

FLAG: NUMBER OF COMMIT ADDRESSES IS NOT EQUAL TO NUMBER OF DATES

DISCREPANCY FOUND AT: https://github.com/nychealth/coronavirus-data/commits/master?after=b671e0ed09b458f148110ad0d71479cf62580ea4+34&branch=master&path%5B%5D=by-race.csv


In [167]:
from urllib.error import HTTPError
def commitChecker(commit):
    # affirms that each commit actually goes to an existing csv. In the case where a given day had multiple commits 
    # for the same file, only one of those commit addresses would work, and so this function cleans our commits of 
    # any non functional addresses
    try:
        html = urlopen('https://raw.githubusercontent.com/nychealth/coronavirus-data/' + commit + '/by-race.csv')
    except HTTPError as e:
        print(e)
        print("bad commit: " + commit)
        return False
    else:
        return True

In [168]:
# clean each list of unnecessary scraped attributes/values
days = [y.text for y in days]

commits = [x['value'] for x in commits]

commits = [c for c in commits if commitChecker(c)]
# applies commitChecker function to commits

# mini test to make sure that the lists for commits and days are now of equal length
print("testing: len(commits) == len(days)")
result = len(commits) == len(days)
print("expected result: True")
print("actual result: " + str(result))
print("matches expected? " + str(result == True))

HTTP Error 404: Not Found
bad commit: 700a357f272b309fd8841b5e7dc4f4a53bb116ff
testing: len(commits) == len(days)
expected result: True
actual result: True
matches expected? True


In [ ]:
for day,commit in zip(days,commits):
    # go through the days and commit values and create proper URLs based on the commit and the file name to create 
    # daily dataframes, which we append to a cumulative dataframe, in this instance for the by-race dataset.
    print(str(day) + ': ' + str(commit))
    df = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/' + commit + '/by-race.csv')
    df.insert(0, 'Date', day)
    masterData = masterData.append(df, ignore_index=True)

In [173]:
masterData.head()

,Date,RACE_GROUP,CASE_RATE_ADJ,HOSPITALIZED_RATE_ADJ,DEATH_RATE_ADJ,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT
0,"Commits on Nov 9, 2020",Asian/Pacific-Islander,922.14,293.56,111.99,12365,3981,1441
1,"Commits on Nov 9, 2020",Black/African-American,1941.90,738.67,249.62,39619,15804,5364
2,"Commits on Nov 9, 2020",Hispanic/Latino,2199.41,743.55,268.58,53542,17478,5995
3,"Commits on Nov 9, 2020",White,1426.05,349.00,125.83,43397,12480,4938
4,"Commits on Nov 8, 2020",Asian/Pacific-Islander,916.27,293.10,111.86,12286,3974,1439


In [175]:
# write the scraped master by-race dataFrame to a csv
with open('./masterData.csv', "w", newline='') as f:    
    writer = csv.writer(f, delimiter=',')
masterData.to_csv ('./masterData.csv', index = False, header=True)

In [176]:
confirmedProbableData = pd.read_csv('confirmedProbableDeathData.csv')
# do the same thing as above for the confirmed-probable dataset. 
# We can use the same day/commit lists for this data set.

In [177]:
for day,commit in zip(days,commits):
    fd = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/' + commit + '/deaths/probable-confirmed-by-race.csv')
    fd.insert(0,'Date',day)
    confirmedProbableData = confirmedProbableData.append(fd,ignore_index=True)

In [180]:
confirmedProbableData.head()

,Date,RACE_GROUP,CONFIRMED_DEATH,PROBABLE_DEATH
0,"Commits on Nov 9, 2020",Hispanic/Latino,5995,1263
1,"Commits on Nov 9, 2020",Black/African-American,5364,1436
2,"Commits on Nov 9, 2020",White,4938,1203
3,"Commits on Nov 9, 2020",Asian/Pacific-Islander,1441,413
4,"Commits on Nov 9, 2020",Other/Unknown,1672,334
...,...,...,...,...
1033,"Commits on May 18, 2020",Black/African-American,4409,1347
1034,"Commits on May 18, 2020",White,4039,1101
1035,"Commits on May 18, 2020",Asian/Pacific-Islander,1220,389
1036,"Commits on May 18, 2020",Other/Unknown,1569,384


In [185]:
with open('./confirmedProbableDeathData.csv', "w", newline='') as f:    
    writer = csv.writer(f, delimiter=',')
masterData.to_csv ('./confirmedProbableDeathData.csv', index = False, header=True)